# Introdocution


The purpose of this notebook is to save a mask for the California Current System that can be used with the EMU tool. 



In [7]:
import sys
from os.path import join,expanduser
user_home_dir = expanduser('~')

#point to the correct src path if this doesnt work
sys.path.insert(0,join(user_home_dir,'efs_ecco/ameza/ESS25-Team_SHERLOCCS/src'))

from src import * 

datadir = lambda x: "/efs_ecco/ameza/ESS25-Team_SHERLOCCS/masks/" + x

In [14]:
## Set top-level file directory for the ECCO NetCDF files
## =================================================================

## currently set to ~/Downloads/ECCO_V4r4_PODAAC, 
## the default if ecco_podaac_download was used to download dataset granules
ECCO_dir = join('/efs_ecco','ameza')

# for access_mode = 's3_open_fsspec', need to specify the root directory 
# containing the jsons
jsons_root_dir = join('/efs_ecco','mzz-jsons')

access_mode = 's3_open_fsspec'

## access datasets needed for this tutorial

ShortNames_list = ["ECCO_L4_GEOMETRY_LLC0090GRID_V4R4"]
StartDate = '1991-01'
EndDate = '2018-12'
ds_GEOM = ea.ecco_podaac_to_xrdataset(ShortNames_list,\
                                            StartDate=StartDate,EndDate=EndDate,\
                                            mode=access_mode,\
                                            download_root_dir=ECCO_dir,\
                                            max_avail_frac=0.5,\
                                            jsons_root_dir=jsons_root_dir)

ccc_mask = xr.open_dataset(datadir("CCC_mask_Anthony.nc"))

In [18]:
def calculate_emu_mask(ds_mask, ds_GEOM): 
    pos_def_mask = enforce_pos_def_mask(ds_mask)
    ocean_volume = get_cell_volumes(ds_GEOM)
    
    masked_volume = ocean_volume * pos_def_mask
    
    return masked_volume / masked_volume.sum()
    
def save_emu_mask(ds_mask, ds_GEOM, fname="mask"):
    emu_mask = calculate_emu_mask(ds_mask, ds_GEOM).compute()
    mask3d_c = ecco.llc_tiles_to_compact(emu_mask, less_output=True)
    print(np.isnan(mask3d_c).sum())

    file_path = datadir(f"{fname}.bin")
    print(f"Saving mask as: {file_path}")
    mask3d_c.astype('>f4').tofile(file_path)

In [19]:
save_emu_mask(ccc_mask["CCC_surf_mask"], ds_GEOM, fname = "CCC_surf_mask")

llc_faces_to_compact: data_compact array shape (50, 1170, 90)
0
Saving mask as: /efs_ecco/ameza/ESS25-Team_SHERLOCCS/masks/CCC_surf_mask.bin


In [20]:
save_emu_mask(ccc_mask["CCC_sub_surf_mask"], ds_GEOM, fname = "CCC_sub_surf_mask")

llc_faces_to_compact: data_compact array shape (50, 1170, 90)
0
Saving mask as: /efs_ecco/ameza/ESS25-Team_SHERLOCCS/masks/CCC_sub_surf_mask.bin
